In [1]:
import re
import math
import numpy as np
import pandas as pd

# PREPROCESSING

In [10]:
from stop_word import stop_words
import nltk
from nltk.stem import PorterStemmer
import re
nltk.download('punkt') # Download data yang dibutuhkan untuk NLTK (jalankan sekali pada setiap instalasi)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv("./dataset/Dataset2.csv") # Memuat dataset

In [3]:
df

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1
...,...,...,...,...
4004,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0
4005,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0
4006,https://www.activistpost.com/2017/09/ron-paul-...,"Ron Paul on Trump, Anarchism & the AltRight",NaN,0
4007,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1


In [30]:
df.dropna(inplace=True) # Drop semua dataset bernilai NaN

In [31]:
df['Full'] = df['Headline'].loc[0:2000] + ' ' + df['Body'].loc[0:2000] # Membuat kolom baru full terdiri dari gabungan kolom headline dan body

In [37]:
df['Full']
df.dropna(inplace=True) # Drop semua dataset bernilai NaN

#### PREPROCESSING FUNCTION

In [32]:
def stemming(word):
    stemmer = PorterStemmer()
    stemmed_word = stemmer.stem(word)
    return stemmed_word

In [33]:
def stopword_removal_and_filtering(words):
    # Menghilangkan stopwords dan Case Folding
    filtered_words = [word for word in words if word.lower() not in stop_words] 

    # Filtering singkatan sederhana
    filtered_words = [re.sub(r'\.', '', word) for word in filtered_words]

    # Filtering angka
    filtered_words = [re.sub(r'\d', '', word) for word in filtered_words]
    
    # Filtering data redundan
    filtered_words = set(filtered_words)
    
    return filtered_words

In [34]:
def preprocessing(text):
    # Menghilangkan tanda baca
    text = re.sub(r'[^\w\s-]', ' ', text)

    # Memisahkan teks menjadi kata-kata (Tokenizing)
    words = text.split()
    
    # Stop word removal dan stemming
    filtered_words = stopword_removal_and_filtering(words)

    # Stemming
    filtered_words = [stemming(i) for i in filtered_words]

    # Menghilangkan spasi ekstra
    processed_text = ' '.join(filtered_words[1:])

    return processed_text


In [38]:
X_dataset_fullset = df['Full'].apply(preprocessing) # Proses preprocessing dataset

In [39]:
X_dataset_fullset = X_dataset_fullset.values.flatten() # Menggabungkan data menjadi 1 dimensi

In [40]:
X_dataset_fullset = X_dataset_fullset.tolist() # Mengubah data menjadi list

In [41]:
X_string_fullset = ' '.join(X_dataset_fullset) # Menggabung data menjadi sebuah string

In [42]:
X_feature_fullset = set(X_string_fullset.split(" ")) # Memisahkan string menjadi fitur - fitur
X_feature_fullset = list(X_feature_fullset)

In [43]:
X_feature_fullset = pd.Series(data=X_feature_fullset) # Mengubah list menjadi data series 

## PERHITUNGAN BOBOT FITUR (TF-IDF)

In [44]:
import re
import math
import numpy as np
import pandas as pd

In [45]:
docs = X_dataset_fullset
processed_unique_words = X_feature_fullset

In [46]:
tf = np.zeros((len(processed_unique_words), len(docs)))

### Menghitung Term Frequency

In [47]:
# for i in range(len(docs)):
#     for x in range(len(processed_unique_words)):
#         count = docs[i].count(processed_unique_words[x])
#         tf[x][i] = 0 if count == 0 else 1 + math.log(1 + count)

In [48]:
# df_tf = pd.DataFrame(tf)

In [49]:
# df_tf.to_csv('tf.csv',index=False) # Menyimpan hasil perhitungan term frequency

## Data Preparation (START RUNNING DARI SINI)

### Load Feature

In [50]:
tfdf = pd.read_csv("./tf.csv")
tfdf.index = processed_unique_words
tfdf = tfdf.transpose()

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [53]:
X_train, X_test, y_train, y_test = train_test_split(tfdf, df['Label'], test_size=0.10, random_state=42) # Data Splitting

# Normalisasi Data
norm_X_train = scaler.fit_transform(X_train)
norm_X_test = scaler.fit_transform(X_test)

## Create Model

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

### Arsitektur 1

In [55]:
model = Sequential()

model.add(Dense(512, input_shape=(norm_X_train.shape[1],), activation='relu'))

# Hidden layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification (fake or not)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

### Arsitektur 2

In [ ]:
model = Sequential()

# Input layer
model.add(Dense(512, input_shape=(norm_X_train.shape[1],), activation='relu'))
model.add(Dropout(0.6))

# Hidden layers
model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.6))

# Additional hidden layer
model.add(Dense(64, activation='relu'))


# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

### Arsitektur 3

In [ ]:
model = Sequential()

model.add(Dense(512, input_shape=(norm_X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))

# Hidden layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Additional hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

### ARSITEKTUR 4

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=2000, output_dim=128, input_length=norm_X_train.shape[1],))

# LSTM layer
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

### Arsitektur 5

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

model = Sequential()

# Input layer
model.add(Dense(512, input_shape=(norm_X_train.shape[1],), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Hidden layers
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

### Arsitektur 6

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU

model = Sequential()

# Input layer
model.add(Dense(1024, input_shape=(norm_X_train.shape[1],), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Hidden layers
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

In [56]:
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

Epoch 1/100
7/7 [==============================] - 2s 150ms/step - loss: 0.5913 - accuracy: 0.6963 - val_loss: 0.3554 - val_accuracy: 0.8547
Epoch 2/100
7/7 [==============================] - 1s 113ms/step - loss: 0.2142 - accuracy: 0.9323 - val_loss: 0.1505 - val_accuracy: 0.9274
Epoch 3/100
7/7 [==============================] - 1s 115ms/step - loss: 0.0598 - accuracy: 0.9832 - val_loss: 0.1558 - val_accuracy: 0.9665
Epoch 4/100
7/7 [==============================] - 1s 114ms/step - loss: 0.0173 - accuracy: 0.9975 - val_loss: 0.1112 - val_accuracy: 0.9777
Epoch 5/100
7/7 [==============================] - 1s 103ms/step - loss: 0.0061 - accuracy: 0.9988 - val_loss: 0.1616 - val_accuracy: 0.9665
Epoch 6/100
7/7 [==============================] - 1s 111ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.1854 - val_accuracy: 0.9721
Epoch 7/100
7/7 [==============================] - 1s 115ms/step - loss: 9.6117e-04 - accuracy: 1.0000 - val_loss: 0.2297 - val_accuracy: 0.9609
Epoch 8/1

In [60]:
loss, accuracy = model.evaluate(norm_X_test, y_test)

7/7 [==============================] - 0s 9ms/step - loss: 0.4745 - accuracy: 0.9598


In [62]:
model.save("my_model.h5")

c:\Users\Asus\anaconda3\envs\py3.11-TF2.0\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Model Jurnal Referensi

In [113]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [115]:
decision_tree = DecisionTreeClassifier(criterion='gini', splitter='best', min_samples_split=2,
                                       min_samples_leaf=1, max_features=None, random_state=42)
random_forest = RandomForestClassifier(n_estimators=100, criterion='gini', max_features='auto', random_state=42)
svm_model = SVC(C=1.0, kernel='poly', degree=1, coef0=1.0, gamma=1.0)
gradient_boosted_trees = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

In [117]:
decision_tree.fit(norm_X_train, y_train)
random_forest.fit(norm_X_train, y_train)
svm_model.fit(norm_X_train, y_train)
gradient_boosted_trees.fit(norm_X_train, y_train)

### Model Testing

In [102]:
def TFIDF(processed_unique_words,docs):
    tf = np.zeros((len(processed_unique_words), len(docs)))
    for i in range(len(docs)):
        for x in range(len(processed_unique_words)):
            count = docs[i].count(processed_unique_words[x])
            tf[x][i] = 0 if count == 0 else 1 + math.log(1 + count)
    return tf

In [119]:
# Contoh data teks (dokumen)
documents = [
    "COVID-19 Cure Found! Scientists have discovered a cure for COVID-19",
    "Breaking: UFO Sightings on the Rise! Multiple reports of UFO sightings around the world",
    "New Study: Chocolate is Healthy! Eating chocolate has been proven to improve health",
    "Government Denies Alien Contact Officials deny any contact with extraterrestrial beings",
    "Fake News Alert! Rumors of a zombie apocalypse are false, authorities confirm."
]

tf = TFIDF(tfdf.columns,documents)
tf = tf.transpose()

In [125]:
predictions = model.predict(tf)
predicted_labels = [df['Label'].iloc[prediction.argmax()] for prediction in predictions]
print(predicted_labels)
for i in predicted_labels:
    if i == 1:
        print("True")
    else:
        print("Hoax bejir")

1/1 [==============================] - 0s 22ms/step
[1, 1, 1, 1, 1]
True
True
True
True
True
